# Task02
Needs Blackhole configured

In [184]:
# Import requirements
import spotipy
import pandas
import random
import numpy
import os

In [185]:
import sounddevice as sd
from scipy.io.wavfile import write

fs = 44100  # Sample rate
seconds = 24  # Duration of recording

In [186]:
# Import specific functions
from lib import *

In [187]:
# subject name/id
subject_id = 'sub-12'

In [188]:
# Setup Spotipy
scope = "user-library-read,user-read-playback-state,user-modify-playback-state"
sp = spotipy.Spotify(
    auth_manager=spotipy.oauth2.SpotifyOAuth(scope=scope,client_id='253a1345850d4a5c94995c1159d286e4',
                                             client_secret='9a03e1bcb8124eed9b661991e84bf477',
                                             redirect_uri='http://127.0.0.1:9090'),
)

# choose deviceID (1st)
res = sp.devices()
devID = res['devices'][0]['id']

## Import subject-specific selection of songs

In [201]:
# read file
D = pandas.read_table(os.path.join('..','data','music_matrices',
                                   f"music_{subject_id}.tsv"),
                                index_col = "Emotion")

# extract list of emotions
emotionList = D.index.values

# Generate two new columns (start1MS and start2MS) in milliseconds for each song
for emotion in emotionList:
    D.loc[emotion,"startMS1"] = (int(D.start1[emotion][1:3])*60 + int(D.start1[emotion][4:6])) * 1000
    D.loc[emotion,"startMS2"] = (int(D.start2[emotion][1:3])*60 + int(D.start2[emotion][4:6])) * 1000

## Version 0.2a - 2 songs coupled by emotion

In [204]:
run_id = 'task-02a_run-1'

# create a new list with the same elements as emotionList, but in a random order
randomEmotionList = emotionList.copy()
random.shuffle(randomEmotionList)

# Initialize Dataframe to record timestamps
timeMatrix = pandas.DataFrame(columns=["emotion","trackID","timestamp0","before_music_start", "after_music_start", "before_noise_start", "after_noise_start"])

# set initial volume
sp.volume(100,devID)

# first baseline
timestamp0 = time.time()
#sp.start_playback(uris=['spotify:track:7Ipe9FfRceDN51YMyd99pO'],position_ms=1000,device_id=devID)
#wait_until(timestamp0 + 2)

# trial loop
for emotion in randomEmotionList:
    print(f"Now playing: {emotion}")

    for i in range(1,3):
        song = D[f"trackID{str(i)}"][emotion]
        start_time = D[f"startMS{str(i)}"][emotion].item()

        print(f"Now playing: {song} starting at {str(start_time)}")

        timestamp1 = time.time()

        sp.start_playback(
            uris=[f'spotify:track:{song}'],
            position_ms=start_time,
            device_id=devID,
        )

        timestamp2 = time.time()

        myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)

        sd.wait()  # Wait until recording is finished
        
        sp.pause_playback()

        write(os.path.join('..','data','wav_files',
                           f'{subject_id}_{emotion.replace(" ", "")}_{song}.wav'),
                         fs, myrecording)  # Save as WAV file

        timestamp3 = time.time()

        wait_until(timestamp3 + 2)

print('Done!')

Now playing: Joyful activation
Now playing: 6m5NwZvZ8qeNVPeF4sSN3A starting at 110000.0
Now playing: 5FiB1uNoGZE4PenzZd7Imu starting at 102000.0
Now playing: Transcendence
Now playing: 7yWT9m9UEI06Be9nb7M5LB starting at 377000.0
Now playing: 4T2PrxnP1BnBe5Kwilx0ul starting at 247000.0


HTTP Error for PUT to https://api.spotify.com/v1/me/player/pause with Params: {} returned 403 due to Player command failed: Restriction violated


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/me/player/pause:
 Player command failed: Restriction violated, reason: UNKNOWN